<h1 align="center"><font size="7" face="arial" color="#DC5A29">Zerlaut Usage</font></h1>

<h1><font size="6" face="arial" color="black">Initialise</font></h1>

In [ ]:
import sys
path_tvb_modify = '.'
sys.path.insert(0, path_tvb_modify)

In [ ]:
%%capture
%matplotlib widget

# Import a bunch of stuff to ease command line usage
from tvb.simulator.lab import *
# FC 
from tvb.datatypes.time_series import TimeSeriesRegion
from tvb.datatypes import graph
# Phase Plane
from tvb.simulator.plot.phase_plane_interactive import PhasePlaneInteractive
# ICA
import scipy.stats
from sklearn.decomposition import FastICA
# brain maps

import numpy as np
import time as tm
import matplotlib.pyplot as plt        
import matplotlib.gridspec as gridspec 

<h1><font size="6" face="arial" color="black">Setting up the simulation</font></h1>

- ## <font size="5" face="arial" color="black"> Connectivity</font></h3>

In [ ]:
# Import the anatomical structural connectivity.
TVB_con = connectivity.Connectivity().from_file()      
nregions = len(TVB_con.region_labels)     #number of regions
TVB_con.speed = np.array(np.inf)                    #set the conduction speed
TVB_con.configure()
TVB_SC = TVB_con.weights

- ## <font size="5" face="arial" color="black">Coupling function</font></h3>

In [ ]:
# Initialise a Coupling function.
G = np.array(1.0)
con_coupling = coupling.Scaling(a=G)

- ## <font size="5" face="arial" color="black">Integrator</font></h3>

In [ ]:
# Initialise an Integrator scheme.
dt = 0.1               #integration steps [ms]

#heunint = integrators.HeunDeterministic(dt=dt)

nsigma = 1e-6         #standard deviation of the noise
hiss = noise.Additive(nsig=np.array([nsigma, nsigma,0,0]))
heunint = integrators.HeunStochastic(dt=dt, noise=hiss)

- ## <font size="5" face="arial" color="black">Monitors</font></h3>

In [ ]:
# Initialise some Monitors with period in physical time.
mon_raw = monitors.Raw()
mon_tavg = monitors.TemporalAverage(period=1) #1000 Hz        

#Bundle them
what_to_watch = (mon_raw, mon_tavg)

***

<h1><font size="6" face="arial" color="black">1. First Order</font></h1>

- ## <font size="5" face="arial" color="black"> Model</font>

In [ ]:
# Initialise a Model.
zerlaut1 = models.ZerlautAdaptationFirstOrder()
zerlaut1

<h1><font size="6"face="arial" color="black">Simulate</font></h1>

In [ ]:
# Initialise the Simulator.
sim = simulator.Simulator(model=zerlaut1,
                          connectivity=TVB_con,
                          conduction_speed=np.float64(TVB_con.speed),
                          coupling=con_coupling,
                          integrator=heunint,
                          monitors=what_to_watch)
sim.configure()

In [ ]:
# Perform the simulation.
tic = tm.time()

raw_data, raw_time = [], []
tavg_data, tavg_time = [], []

for raw, tavg in sim(simulation_length=10000.):   
    if not raw is None:
        raw_time.append(raw[0])
        raw_data.append(raw[1])
        
    if not tavg is None:
        tavg_time.append(tavg[0])                 
        tavg_data.append(tavg[1])
                
'simulation required %0.3f seconds.' % (tm.time()-tic)

<h2><font size="4"face="arial" color="black">Visualize simulation</font></h2>

In [ ]:
# Make the lists numpy.arrays for easier use.    
RAW = np.squeeze(np.array(raw_data))
TAVG = np.squeeze(np.array(tavg_data))

In [ ]:
RAW/=np.max(RAW)
TAVG/=np.max(TAVG)

In [ ]:
# Plot the raw time series 
no_regions = 10
fig1 = plt.figure(figsize=(10,8))
plt.plot(raw_time[:], RAW[:, :no_regions] + np.arange(0,10,1))
plt.title('Raw Neuronal Activity', fontsize=20)
plt.xlabel('Time [ms]', fontsize=20)
plt.yticks(range(no_regions), TVB_con.region_labels[:no_regions], fontsize=10)

# Plot the temporally averaged time series
fig2 = plt.figure(figsize=(10,8))
plt.plot(tavg_time[:], TAVG[:, :no_regions] + np.arange(0,10,1))
plt.title('Temporally Averaged Neuronal Activity', fontsize=20)
plt.xlabel('Time [ms]', fontsize=20)
plt.yticks(range(no_regions), TVB_con.region_labels[:no_regions], fontsize=10)

plt.show()

<h2><font size="6" face="arial" color="black">Exploring a Model</font></h2>

- ## <font size="5" face="arial" color="black">Create an interactive phase-plane</font></h3>

In [ ]:
# Initialise an Integrator scheme.
dt = 1    #integration steps [ms]
heunint = integrators.HeunDeterministic(dt=dt)

In [ ]:
# Create and lunch the phase-plane tool.
ppi_fig = PhasePlaneInteractive(model=zerlaut1, integrator=heunint)
ppi_fig.show()

- ## <font size="5" face="arial" color="black">Using a stochastic integration scheme</font></h3>

In [ ]:
# Initialise an Integrator scheme.
dt = 1                                           #integration steps [ms]
heunstochint = integrators.HeunStochastic(dt=dt)
heunstochint.noise.nsig = np.array([1.e-6])                  #standard deviation of the noise 

In [ ]:
ppi_fig = PhasePlaneInteractive(model=zerlaut1, integrator=heunstochint)
ppi_fig.show()

***

<h1><font size="6" face="arial" color="black">1. Second Order</font></h1>

- ## <font size="5" face="arial" color="black"> Model</font>

In [ ]:
# Initialise a Model.
zerlaut2 = models.ZerlautAdaptationSecondOrder()
zerlaut2

<h1><font size="6"face="arial" color="black">Simulate</font></h1>

In [ ]:
# Initialise the Simulator.
sim = simulator.Simulator(model=zerlaut2,
                          connectivity=TVB_con,
                          conduction_speed=np.float64(TVB_con.speed),
                          coupling=con_coupling,
                          integrator=heunint,
                          monitors=what_to_watch)
sim.configure()

In [ ]:
# Perform the simulation.
tic = tm.time()

raw_data, raw_time = [], []
tavg_data, tavg_time = [], []

for raw, tavg in sim(simulation_length=10000.):   
    if not raw is None:
        raw_time.append(raw[0])
        raw_data.append(raw[1])
        
    if not tavg is None:
        tavg_time.append(tavg[0])                 
        tavg_data.append(tavg[1])
                
'simulation required %0.3f seconds.' % (tm.time()-tic)

<h2><font size="4"face="arial" color="black">Visualize simulation</font></h2>

In [ ]:
# Make the lists numpy.arrays for easier use.    
RAW = np.squeeze(np.array(raw_data))
TAVG = np.squeeze(np.array(tavg_data))

In [ ]:
RAW/=np.max(RAW)
TAVG/=np.max(TAVG)

In [ ]:
# Normalize the time series
raw_data /= (np.max(raw_data, 0) - np.min(raw_data, 0))
tavg_data /= (np.max(tavg_data, 0) - np.min(tavg_data, 0))

In [ ]:
# Plot the raw time series 
no_regions = 10
fig1 = plt.figure(figsize=(10,8))
plt.plot(raw_time[:], RAW[:, :no_regions] + np.r_[:no_regions])
plt.title('Raw Neuronal Activity', fontsize=20)
plt.xlabel('Time [ms]', fontsize=20)
plt.yticks(range(no_regions), TVB_con.region_labels[:no_regions], fontsize=10)

# Plot the temporally averaged time series
fig2 = plt.figure(figsize=(10,8))
plt.plot(tavg_time[:], TAVG[:, :no_regions] + np.r_[:no_regions])
plt.title('Temporally Averaged Neuronal Activity', fontsize=20)
plt.xlabel('Time [ms]', fontsize=20)
plt.yticks(range(no_regions), TVB_con.region_labels[:no_regions], fontsize=10)

plt.show()

<h2><font size="6" face="arial" color="black">Exploring a Model</font></h2>

- ## <font size="5" face="arial" color="black">Create an interactive phase-plane</font></h3>

In [ ]:
# Initialise an Integrator scheme.
dt = 1    #integration steps [ms]
heunint = integrators.HeunDeterministic(dt=dt)

In [ ]:
# Create and lunch the phase-plane tool.
ppi_fig = PhasePlaneInteractive(model=zerlaut2, integrator=heunint)
ppi_fig.show()

- ## <font size="5" face="arial" color="black">Using a stochastic integration scheme</font></h3>

In [ ]:
# Initialise an Integrator scheme.
dt = 1                                           #integration steps [ms]
heunstochint = integrators.HeunStochastic(dt=dt)
heunstochint.noise.nsig = np.array([1.e-6])                  #standard deviation of the noise 

In [ ]:
ppi_fig = PhasePlaneInteractive(model=zerlaut2, integrator=heunstochint)
ppi_fig.show()